### 一、BigQuery中 SQL脚本对于 unixtime，timestamp，datetime的处理过程
###### 1、理解三者在bigquery中的样式

In [ ]:
# 1.1 数据库中存放的unixtime通常是 int64/bigint类型  比32位更加难以溢出
# 比如 1685983854000 代表的是从1970年1月1日午夜开始经过的秒数
# unixtime有秒，毫秒，微妙的区分 整数的长度和对应的解析函数也会不同 分别是10,13,16位数 

# 1.2 timestamp通常是timestamp类型 与datetime的区别是带时区信息 即所有的时间传入后自动根据时区信息改为UTC时间再存放
# 比如 2023-05-10 22:57:16.840000 UTC

# 1.3 datetime通常是datetime类型 不包含时区信息 传入的时间是什么就是什么 不做加工
# 比如 2023-05-11T06:57:16.840000

### 一、BigQuery中 SQL脚本对于 unixtime，timestamp，datetime的处理过程
###### 2、BigQuery 从int64的unixtime 翻译出一个本地时区的 datetime
（1） 从unixtime变为timestamp类型 （2） 从timestamp类型转变为datetime

In [ ]:
# 2.1 从unixtime变为timestamp类型

# bigquery中存在 timestamp_seconds,timestamp_millis,timestam_micros三个转化函数int -> timestamp

second = 1685983831
print('second时间戳的长度是：',len(str(second)), '，代表这个是秒，请使用如下sql')
# 输入：select timestamp_seconds(1685983831)
# 输出：2023-06-05 16:50:31 UTC

millisecond = 1685983831534
print('millisecond时间戳的长度是：',len(str(millisecond)), '，代表这个是毫秒，请使用如下sql')
# 输入：select timestamp_millis(1685983831534)
# 输出：2023-06-05 16:50:31.534000 UTC

microsecond = 1685983831534679
print('microsecond时间戳的长度是：',len(str(microsecond)), '，代表这个是微秒，请使用如下sql')
# 输入：select timestamp_micros(1685983831534679)
# 输出：2023-06-05 16:50:31.534679 UTC

In [ ]:
# 2.2 从timestamp类型转变为datetime类型

# 从上面拿到了timestamp 解析下他的构成
# 2023-06-05 16:50:31.534679 UTC -> %Y-%m-%d %H-%M-%S.%f 
# .%f  无论长短都可以容纳进去 也即毫秒 微妙都是用这个代表

# 接下来转为datetime 这个过程可以进行时区加工 得到对应某个时区的时间
# datetime函数（参数一 timestamp类型/字符串类型, 参数二是时区）
# 输入：select datetime(timestamp_micros(1685983831534679),'Asia/Shanghai')
# 输出：2023-06-06T00:50:31.534679 -> %Y-%m-%d %H-%M-%S.%f 


### 一、BigQuery中 SQL脚本对于unixtime，timestamp，datetime的处理过程
###### 3、BigQuery 一个本地时区的 datetime翻译出一个int64的unixtime
（1） datetime类型变为timestamp （2）timestamp类型变为unixtime

In [ ]:
# 3.1 从datetime类型变为timestamp
# timestamp函数（参数一 datetime类型/字符串类型，参数二是时区）
# 输入：select timestamp(datetime('2023-06-06T00:50:31.534679'),'Asia/Shanghai')
# 输出：2023-06-05 16:50:31.534679 UTC
# 基于时区还原为utc，不提供就意味着当做utc处理

In [ ]:
# 3.2 从timestamp类型变为unixtime
# 一样存在三个函数 分别对应 秒 毫秒 微秒

# 产出秒
# 输入：select unix_seconds(timestamp('2023-06-05 16:50:31.534679 UTC'))
# 输出：1685983831

# 产出毫秒
# 输入：select unix_millis(timestamp('2023-06-05 16:50:31.534679 UTC'))
# 输出：1685983831534

# 产出微秒
# 输入：select unix_micros(timestamp('2023-06-05 16:50:31.534679 UTC'))
# 输出：1685983831534679


### 二、MYSql中 SQL脚本对于 unixtime，timestamp，datetime的处理过程
###### 1、理解三者在mysql中的样式

###### UNIXTIME与bigquery是一致的，但是timestamp和datetime差别巨大，
1. 首先这两个东西在mysql中都不携带时区信息，也即timestamp不会有后缀的UTC
2. timestamp这个数据类型会根据MYSQL系统所处的时间改变每次查询的结果值；datetime则不会
3. timestamp存储的时间范围是：1970-01-01 00:00:01 UTC 到 2038-01-19 03:14:07 UTC 的日期和时间；   
  datetime存储的时间范围是：1000-01-01 00:00:00 到 9999-12-31 23:59:59。    
  后者明细范围更广，前者还可能因为时区出错，对于需要稳定不出错好解释的还是用int64存放unixtime最好 
4. 其次翻译unixtime用的from_unixtime是会依据MYSQL系统所处的时区对于翻译过来的timestamp执行时区加工的，所以不一定就是UTC   
  为了避免这个问题，建议在查询时写SET time_zone = '+00:00';这个会改本次查询的时区，之后再写SELECT FROM_UNIXTIME(unix_timestamp);


### 二、MYSql中 SQL脚本对于 unixtime，timestamp，datetime的处理过程
###### 2、 转化过程

1.mysql没有转化为timestamp的函数当你把一个unixtime放入一列timestamp类型时，内部将会翻译为UTC时间存放，但是查询的时候会自动的加上MYSQL端本时区的信息   
2.翻译时间戳为可读的日期，使用的函数是使用from_unixtime   
输入：select from_unixtime(1778852390)   
输出：2026-05-15 13:39:50    
3.反过来，datetime/timestamp变为unixtime可以用UNIX_TIMESTAMP()      
输入：select UNIX_TIMESTAMP('2026-05-15 13:39:50')    
输出：1778852390     
4.以上的过程中并没有能让用户自定义时区的部分，所以过去的处理中采取直接在unix时间戳上面加对应的秒数来解决时区问题   
5.另外如果要转13和16的unix，需要除以1000，在将精度部分拼接回去，如下：   
输入：select CONCAT(FROM_UNIXTIME(1234567891987 DIV 1000), '.', RPAD(1234567891987 MOD 1000, 6, '0'))   
输出：2009-02-13 23:31:31.987000

### 三、Python中 timestamp(就是unix时间戳)，datetime（分为naive和aware）的处理过程 python没有mysql那种意义上的timestamp
###### 1、 演示生成unix，生成datetime（分为aware和非aware，即带时区和隐藏时区）

In [ ]:
import datetime
import time
import pytz

# 1.1 获取下现在的unixtime 等于mysql中的int，float
print(1.1)
now_unix = time.time()
print(f'现在的UN是：{now_unix}') 
print(f'通过time.time()生成的值是{now_unix}，数据类型是：{type(now_unix)}') # 高精度的浮点数，即unixtime
print()

# 1.2 获取下现在的utc datetime aware版本的 等于mysql中的datetime 
print(1.2)
now_datetime_utc1 = datetime.datetime.now(tz = datetime.timezone.utc) # 这个用了datetime.timezone下的utc对象
now_datetime_utc2 = datetime.datetime.now(tz = pytz.utc)              # 这个用了pytz三方库的utc对象
print(f'现在的UTC时间1是：{now_datetime_utc1}')
print(f'现在的UTC时间2是：{now_datetime_utc2}')
print(f'通过datetime.datetime.now(tz = datetime.timezone.utc)生成的值是{now_datetime_utc1}，数据类型是：{type(now_datetime_utc1)}') # datetime.datetime
print()

# 1.3 获取下现在的北京时区 datetime aware版本的 等于mysql中的datetime 
print(1.3)
beijing_tz = pytz.timezone('Asia/Shanghai')
now_datetime_beijing = datetime.datetime.now(tz = beijing_tz)
print(f'现在的北京时间是：{now_datetime_beijing}')
print(f'通过datetime.datetime.now(tz = beijing_tz)生成的值是{now_datetime_beijing}，数据类型是：{type(now_datetime_beijing)}') # datetime.datetime
print()

# 1.4 获取本地系统默认的时区的datetime 也即非aware版本的 等于mysql中的datetime 
print(1.4)
now_datetime_local = datetime.datetime.now()
print(f'现在的北京时间是：{now_datetime_local}')
print(f'通过datetime.datetime.now()生成的值是{now_datetime_local}，数据类型是：{type(now_datetime_local)}') # datetime.datetime
print('一个明显的区别是，后面没有+08:00的标记，这代表非aware是隐藏时区的时间，通常被视为是本地时间')

### 三、Python中 timestamp(就是unix时间戳)，datetime（分为naive和aware）的处理过程 python没有mysql那种意义上的timestamp
###### 2、 演示unix和datetime.datetime之间的转化
（1） unix -> datetime.datetime （2） datetime.datetime -> unix

In [ ]:
import datetime
import time
import pytz

# 1.1 展示unix -> datetime.datetime ,不给出时区信息，读取本地系统的非aware datetime.datetime生成
print(1.1)
now_unix = time.time()
print(f'现在的UNIX是：{now_unix}') 

now_datetime_local = datetime.datetime.fromtimestamp(now_unix)
print(f'现在的UNIX转化出来的本地时间是：{now_datetime_local}')
print(f'无疑{now_datetime_local}没有后面的UTC标识说明这是一个隐藏时区的，读取本地系统时区的datetime.datetime')
print()

# 1.2 展示unix -> datetime.datetime ,给出UTC时区信息，生成明确 aware 的 datetime.datetime
print(1.2)
now_unix = time.time()
print(f'现在的UNIX是：{now_unix}')

now_datetime_utc = datetime.datetime.fromtimestamp(now_unix,datetime.timezone.utc)
print(f'现在的UNIX转化出来的UTC时间是：{now_datetime_utc}')
print(f'无疑{now_datetime_utc}存在UTC标识说明这是一个显性时区的，转换时用了提供的utc时区生成的datetime.datetime')
print()

# 1.3 展示unix -> datetime.datetime，给出上海时区信息，生成明确 aware 的 datetime.datetime
print(1.3)
now_unix = time.time()
print(f'现在的UNIX是：{now_unix}')

now_datetime_shanghai = datetime.datetime.fromtimestamp(now_unix,pytz.timezone('Asia/Shanghai'))
print(f'现在的UNIX转化出来的上海时间是：{now_datetime_shanghai}')
print(f'无疑{now_datetime_shanghai}存在UTC标识说明这是一个显性时区的，转换时用了提供的上海时区生成的datetime.datetime')
print()

print(f'特别注意，在timestamp中utc前面的日期时间是utc后的，如果要当地时间，必须执行+UTC的时区差，意味着他们连在一起才是完整的时区\n但是python中的datetime.datetime对象，他的时间就是前面的部分，后面的utc只是个标识，告诉你他是哪个时区，他已经计算过了， 不是utc时间')

In [2]:
import datetime
import time
import pytz

# 2.1 先学习如何去除时区信息，使对象变为naive
print(2.1)
now_datetime_NY_aware = datetime.datetime.now(pytz.timezone('America/New_York'))
now_datetime_NY_naive = now_datetime_NY_aware.replace(tzinfo=None)# 只会去除时区信息，但是不会改变任何其他部分
now_datetime_NY_utc = now_datetime_NY_aware.replace(tzinfo=datetime.timezone.utc)
print('aware的纽约datetime：',now_datetime_NY_aware)
print('naive的纽约datetime：',now_datetime_NY_naive)
print('只改时区信息的错误时间，改时区信息不会真的改变时间为对应时区，正确的做法看下面转化时区部分：',now_datetime_NY_utc)
print()

# 2.2 展示datetime.datetime -> unix 
# 方法一：datetime.datetime对象.timestamp()
# 方法二：datetime.datetime.timestamp(datetime.datetime对象)

print(2.2)
print(f'现在的时间戳是：',time.time())
# 情况一 aware的datetime.datetime ，比如上面的now_datetime_NY_aware
print(f'aware纽约时间转回去时间戳{now_datetime_NY_aware.timestamp()}')# 不接受参数，所以时区信息必须给出，自动传过去
# 情况二 naive的datetime.datetime， 比如上面的now_datetime_NY_naive
print(f'naive纽约时间转回去时间戳是错误的，因为缺乏时区信息传递了本地的时区过去{now_datetime_NY_naive.timestamp()}')
print()

2.1
aware的纽约datetime： 2024-02-20 04:49:26.073083-05:00
naive的纽约datetime： 2024-02-20 04:49:26.073083
只改时区信息的错误时间，改时区信息不会真的改变时间为对应时区，正确的做法看下面转化时区部分： 2024-02-20 04:49:26.073083+00:00

2.2
现在的时间戳是： 1708422566.074101
aware纽约时间转回去时间戳1708422566.073083
naive纽约时间转回去时间戳是错误的，因为缺乏时区信息传递了本地的时区过去1708375766.073083



### 三、Python中 timestamp(就是unix时间戳)，datetime（分为naive和aware）的处理过程 python没有mysql那种意义上的timestamp
###### 3、时区转化处理正确流程

###### 在处理时区转换问题上，最为合理的做法是：
1.对于naive的datetime.datetime对象增加时区信息，比如pytz.timezone(zone = '').localize(datetime.datetime对象) 或者使用datetime.datetime对象.replace(tzinfo=datetime.timezone.utc)赋予时区信息     
2.然后在使用.timestamp()将其还原为unix时间戳    
3.最后根据需要的时区，执行datetime.datetime.fromtimestamp(unix时间戳，tz = '')拿到需要的时区时间     

In [ ]:
pytz.timezone??

In [ ]:
import datetime
import pytz

# 解析时间字符串为无时区的datetime对象
date = datetime.datetime.strptime('2023-05-11 06:57:16.840', "%Y-%m-%d %H:%M:%S.%f")

# 设置时区
utc_tz = pytz.utc # UTC时区对象
new_york_tz = pytz.timezone(zone = 'America/New_York') # 纽约时区对象
shanghai_tz = pytz.timezone('Asia/Shanghai')    # 北京时区对象

# 本地化datetime对象到某个时区
date_utc = utc_tz.localize(date)
date_new_york = new_york_tz.localize(date)
date_shanghai = shanghai_tz.localize(date)

# 展示转化后的，前面都长得一样说明都是UTC时间 后面不一样说明时区反应在后面，真正的时间需要你自己计算下的
print(date_utc)      # 2023-05-11 06:57:16.840000+00:00
print(date_new_york) # 2023-05-11 06:57:16.840000-04:00
print(date_shanghai) # 2023-05-11 06:57:16.840000+08:00
print(date)          # 2023-05-11 06:57:16.840000  # 不含时区信息

# 获取相应的时间戳
print(date_utc.timestamp())      # utc时区下的时间戳
print(date_new_york.timestamp()) # 纽约时区下的时间戳
print(date_shanghai.timestamp()) # 这个是北京时区和最下面的不含时区信息的是一个时间戳，证明了按照本地系统换算
print(date.timestamp()) # 不包含时区的datetime转化时间戳 采取系统本地的时间戳 导致了换区域换服务器会变时间 不能这么操作

# 携带时区的数据转化为字符串
date_str = datetime.datetime.strftime(date_new_york, "%Y-%m-%d %H:%M:%S.%f %z")
print(date_str)

In [ ]:
from datetime import datetime
import pytz

def to_utc_with_offset(time_str, timezone_str):
    # 解析时间字符串为datetime对象，没有时区信息
    local_time = datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S.%f')
    
    # 为时间对象指定时区
    local_tz = pytz.timezone(timezone_str)
    local_time_with_tz = local_tz.localize(local_time, is_dst=None)
    
    # 转换为UTC时间
    utc_time = local_time_with_tz.astimezone(pytz.utc)
    
    # 获取时区偏移(以秒为单位)，转换为小时
    offset_hours = local_time_with_tz.utcoffset().total_seconds() / 3600
    
    # 返回UTC时间和偏移量
    return utc_time.replace(tzinfo=None), offset_hours

# 使用函数
time_str = '2023-05-11 06:57:16.840'
timezone_str = 'Asia/Shanghai'
utc_time, offset_hours = to_utc_with_offset(time_str, timezone_str)

# 打印结果
print(f"UTC时间: {utc_time} 时区偏移: {int(offset_hours)}")